Road space evolution
====================
Reconstructs the road space allocation in a given year

In [1]:
import os
import pandas as pd
import geopandas as gpd
import snman
from snman.constants import *

PERIMETER = '_debug'

# Set these paths according to your own setup
data_directory = os.path.join('C:',os.sep,'Users','lballo','polybox','Research','SNMan','SNMan Shared','data_v2')
inputs_path = os.path.join(data_directory, 'inputs')
process_path = os.path.join(data_directory, 'process', PERIMETER)

export_path = os.path.join(data_directory, 'roadspace_evolution', '_debug')

CRS_internal = 29119    # for Boston
#CRS_internal = 2056      # for Zurich

CRS_for_export = 4326

C:\Users\lballo\AppData\Roaming\Python\Python39\site-packages\osmnx\utils.py:192: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  warnings.warn(


Loading data
------------
Loads the prepared simplified street graph, as well as other datasets such as perimeters, rebuilding regions and measurement regions

In [2]:
print('Load street graph')
G = snman.io.load_street_graph(
    os.path.join(process_path, 'street_graph_edges.gpkg'),
    os.path.join(process_path, 'street_graph_nodes.gpkg'),
    crs=CRS_internal
)
print('Load measurement regions')
measurement_regions_gdf = snman.io.load_measurement_regions(
    os.path.join(inputs_path, 'measurement_regions', 'measurement_regions.gpkg'),
    crs=CRS_internal,
    only_active=False,
    set_filter='cambridge_hex_grid'
)

Load street graph
Load measurement regions


In [3]:
lane_updates_cleaned_gpd = snman.io.import_geofile_to_gdf(
    os.path.join(export_path, 'lane_updates_cleaned.gpkg'),
    crs=CRS_internal
)

# keep only major cycling infra
# excluding shared streets and shared lane pavement markings
lane_updates_cleaned_gpd = lane_updates_cleaned_gpd.query('BIKE_FAC not in [2,6,7]')

In [4]:
#G = snman.street_graph.filter_lanes_by_modes(G, [MODE_PRIVATE_CARS, MODE_TRANSIT], delete_empty_edges=False)

In [5]:
path = os.path.join(inputs_path, 'usa', 'cambridge', 'BikeFacilities_by_year', 'metadata', 'BIKE_FAC Lookup.csv')
bike_fac_names = pd.read_csv(path)

In [6]:
# IMPORT CYCLING FACILITIES IN EACH YEAR AND CREATE THE LN_DESC KEY FOR EVERY YEAR
path = os.path.join(inputs_path, 'usa', 'cambridge', 'BikeFacilities_by_year', 'geojson_selected')
lane_updates = gpd.GeoDataFrame()

lane_keys = {}

for filename in os.listdir(path):
    
    file_path = os.path.join(path, filename)
    if not os.path.isfile(file_path):
        continue
    bike_facilities = snman.io.import_geofile_to_gdf(file_path, crs=CRS_internal)
    #bike_facilities = bike_facilities.query("BIKE_FAC in [1,5,4,8,9,11,12,41,42,81,82,83,84,92,91]")
    
    lane_update = gpd.GeoDataFrame(bike_facilities.geometry)
    # extract the year from the file name
    year = int(filename[14:18])
    lane_update['from_year'] = year
    lane_update['to_year'] = year
    lane_keys[year] = 'steps_' + str(year)
    lane_update['add_lanes'] = lane_update.apply(lambda row: 'L-', axis=1)
    lane_update['BIKE_FAC'] = bike_facilities['BIKE_FAC']
    lane_updates = pd.concat([lane_updates, lane_update], ignore_index=True)
    

# prepare the resulting lane updates
import shapely as shp
lane_updates.geometry = lane_updates.geometry.apply(
    lambda geom: shp.ops.linemerge(
        snman.geometry_tools.ensure_multilinestring(geom)
    )
)
gdf_lines = lane_updates.explode()
gdf_lines.reset_index(drop=True, inplace=True)

C:\Users\lballo\AppData\Local\Temp\ipykernel_6856\634994762.py:33: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  gdf_lines = lane_updates.explode()


In [7]:
lane_updates = pd.merge(lane_updates, bike_fac_names, on='BIKE_FAC', how='left').reset_index(drop=True)
lane_updates = gpd.GeoDataFrame(lane_updates)
snman.io.export_gdf(lane_updates, os.path.join(export_path, 'lane_updates.gpkg'))

In [8]:
# MATCH THE LANE UPDATES ONTO THE STREET GRAPH

import copy
import geopandas as gpd
import shapely as shp

for year, lane_key in lane_keys.items():
    
    print(year)
    
    for uvk, data in G.edges.items():
        data[lane_key] = copy.deepcopy(data[KEY_LANES_DESCRIPTION])
    
    lanes = gpd.GeoDataFrame(lane_updates_cleaned_gpd.query(f"from_year <= {year} <= to_year"))
    
    lanes_opposite = copy.deepcopy(lanes)
    lanes_opposite = lanes_opposite.query('oneway == False')
    lanes_opposite.geometry = lanes_opposite.geometry.apply(
        lambda geom: shp.ops.substring(geom, 1, 0, normalized=True)
    )
    
    lanes = gpd.GeoDataFrame(pd.concat([lanes, lanes_opposite], ignore_index=True), crs=lanes.crs)

    """
    snman.enrichment.match_lane_edits(G, lanes,
        remove_short_overlaps=True,
        remove_sidetrips=False,
        max_dist=30, max_dist_init=30, max_lattice_width=5,
        lanes_key=lane_key
    )
    """
    
    snman.enrichment.match_linestrings(
        G, lanes,
        [{'source_column': 'BIKE_FAC', 'target_column': '_factype_' + lane_key, 'agg': 'list'}],
        remove_short_overlaps=True,
        remove_sidetrips=False,
        max_dist=400, max_dist_init=100, max_lattice_width=5,
        _save_map=(os.path.join(export_path, '_matching_edges.gpkg'), os.path.join(export_path, '_matching_nodes.gpkg'))
    )
    

2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023


In [9]:
# CALCULATE CHANGE ON EDGES
if 1:
    for uvk, data in G.edges.items():
        
        previous_lane_key = None
        for year, lane_key in lane_keys.items():
            
            if previous_lane_key is None:
                data['_bike_fac_change_' + lane_key] = 0
                
            else:
            
                bike_facilities_previous_year = set(
                    data['_factype_' + previous_lane_key + '_forward'] + 
                    data['_factype_' + previous_lane_key + '_backward']
                )
                
                bike_facilities_this_year = set(
                    data['_factype_' + lane_key + '_forward'] + 
                    data['_factype_' + lane_key + '_backward']
                )
                
                if bike_facilities_this_year != bike_facilities_previous_year:
                    data['_bike_fac_change_' + lane_key] = 1
                else:
                    data['_bike_fac_change_' + lane_key] = 0
                
            previous_lane_key = lane_key
            
    edges_df = snman.oxc.graph_to_gdfs(G, nodes=False).assign(x=1)
    timeline_df = pd.DataFrame({'year': lane_keys.keys(), 'lanes_key': lane_keys.values()}).assign(x=1)
    
    changes_df = pd.merge(edges_df, timeline_df, on='x')
    
    changes_df['change'] = changes_df.apply(lambda row: row['_bike_fac_change_steps_' + str(row['year'])], axis=1)
    
    changes_df = changes_df.query('change==1')
    
    changes_df = gpd.GeoDataFrame(changes_df[['osmid', 'year', 'lanes_key', 'change', 'geometry']])
    
    snman.io.export_gdf(changes_df, os.path.join(export_path, 'edge_changes.gpkg'))

In [10]:
if 0:
    def cycling_lanes_length(lanes_key):
        L = snman.lane_graph.create_lane_graph(H, lanes_key)
        return sum(
            [
                e['length']
                for uvk, e
                in L.edges.items()
                if e['lanetype'] == LANETYPE_CYCLING_LANE and e['width'] > 0
            ]
        ) / 1000
    
    def car_lanes_length(lanes_key):
        L = snman.lane_graph.create_lane_graph(H, lanes_key)
        return sum(
            [
                e['length']
                for uvk, e
                in L.edges.items()
                if e['lanetype'] == LANETYPE_MOTORIZED and e['width'] > 0
            ]
        ) / 1000

In [11]:
# GENERATE METRICS
if 0:
    import pandas as pd
    import copy
    
    timeline_df = pd.DataFrame()
    
    for name, data in measurement_regions_gdf.iterrows():
        
        polygon = data.geometry
        H = snman.oxc.truncate.truncate_graph_polygon(G, polygon, quadrat_width=100, retain_all=True)
        
        timeline_df_one_region = pd.DataFrame({'year': lane_keys.keys(), 'lanes_key': lane_keys.values()})
        timeline_df_one_region['measurement_region_name'] = name
        timeline_df_one_region['cycling_lanes_length'] = timeline_df_one_region['lanes_key'].apply(cycling_lanes_length)
        timeline_df_one_region['change_metric'] = timeline_df_one_region['cycling_lanes_length'].diff().fillna(0)
        
        #print(timeline_df_one_region)
        
        timeline_df = pd.concat([timeline_df, timeline_df_one_region], axis=0, ignore_index=True)
        
        timeline_gdf = gpd.GeoDataFrame(
            pd.merge(timeline_df, measurement_regions_gdf, left_on='measurement_region_name', right_index=True, how='left')
        )  
        
        snman.io.export_gdf(timeline_gdf, os.path.join(export_path, 'timeline.gpkg'))

Export
------
Saves the resulting datasets to the disk. Use the *snman_detailed.qgz* file to view them in QGIS.

In [12]:
if 1:
    print('Export network without lanes')
    snman.io.export_street_graph(
        G,
        os.path.join(export_path, 'street_graph_edges.gpkg'),
        os.path.join(export_path, 'street_graph_nodes.gpkg'),
        crs=CRS_for_export,
        lane_keys=[KEY_LANES_DESCRIPTION, KEY_GIVEN_LANES_DESCRIPTION, KEY_LANES_DESCRIPTION_AFTER] + list(lane_keys.values())
    )

if 0:
    print('Export lane geometries')
    SCALING = 1
    snman.io.export_street_graph_with_lanes(
        G,
        list(lane_keys.values()),
        os.path.join(export_path, 'lane_geometries.shp'),
        scaling=SCALING,
        crs=CRS_for_export
    )

Export network without lanes
